In [4]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///stock_copy.db', echo = True)
#engine = create_engine('sqlite:///:memory:', echo = False)
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()
from sqlalchemy import Column, Integer, String, Float, Date, DateTime, SmallInteger, Boolean
from sqlalchemy import ForeignKey
from sqlalchemy.sql import select, text
from sqlalchemy.orm import relationship
from datetime  import datetime
import pandas as pd
import numpy as np
import os

In [5]:
class StockData(Base):
    """銘柄の情報を持つクラス"""
    __tablename__ = 'stock'
    code = Column(Integer, primary_key=True)  # 銘柄コード
    name = Column(String(64), nullable=False)  # 銘柄の名前
    category_id = Column(Integer, nullable=True, index=True)  # 業種
    activated = Column(Boolean, default=False)  # backtest対象か否か
    created_at = Column(DateTime, default=datetime.now())  # 作成日時
    updated_at = Column(DateTime, default=datetime.now(), onupdate=datetime.now())  # 更新日時
    
    history = relationship('History',back_populates='stock')

class History(Base):
    """価格情報を持つクラス"""
    __tablename__ = 'history'
    code = Column(Integer,ForeignKey('stock.code'), primary_key=True, )  # 銘柄コード
    date = Column(Date, primary_key=True, unique=False)  # 日付
    #raw_close_price = Column(Float, nullable=False)  # 終値（株式分割調整前）
    Open = Column(Float, nullable=False)  # 始値
    High = Column(Float, nullable=False)  # 高値
    Low = Column(Float, nullable=False)  # 安値
    Close = Column(Float, nullable=False)  # 終値
    Volume = Column(Integer, nullable=False)  # 出来高
    
    stock = relationship('StockData', back_populates='history')
    
    
#StockData.code = relationship("History", order_by=History.code, back_populates="stock")
Base.metadata.create_all(engine)

2018-02-02 12:58:43,168 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-02-02 12:58:43,176 INFO sqlalchemy.engine.base.Engine ()
2018-02-02 12:58:43,178 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-02-02 12:58:43,179 INFO sqlalchemy.engine.base.Engine ()
2018-02-02 12:58:43,182 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stock")
2018-02-02 12:58:43,183 INFO sqlalchemy.engine.base.Engine ()
2018-02-02 12:58:43,223 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("history")
2018-02-02 12:58:43,225 INFO sqlalchemy.engine.base.Engine ()


In [6]:
files = os.listdir('dd')
for file in files:
    if(os.path.isfile('dd/' + file)):
        hist = pd.read_csv('dd/' + file,
                           names=["date", "Open", "High", "Low", "Close", "Volume"],
                           parse_dates=[0], usecols=range(6))
        stk = pd.read_csv('dd/' + 'info' + '/' + file, names=["code", "name", "category_id"])
        codes = np.repeat(np.array(stk["code"]), (len(hist["date"])))
        hist['code'] = codes
        lst = hist.columns.tolist()
        lst.remove('code')
        lst.insert(0, 'code')
        hist = hist[lst]
        # バックテスト対象,作成日時,更新日時のパラメータの追加
        stk['activated'] = True
        stk['created_at'] = datetime.now()
        stk['updated_at'] = datetime.now()
        try:
            stk.to_sql('stock', engine, index=False, if_exists='append')
            hist.to_sql('history', engine, index=False, if_exists='append')
        except:
            pass

2018-02-02 12:58:43,854 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stock")
2018-02-02 12:58:43,857 INFO sqlalchemy.engine.base.Engine ()
2018-02-02 12:58:43,861 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-02-02 12:58:43,863 INFO sqlalchemy.engine.base.Engine INSERT INTO stock (code, name, category_id, activated, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?)
2018-02-02 12:58:43,865 INFO sqlalchemy.engine.base.Engine (1301, '(株)極洋', '水産・農林業', 1, '2018-02-02 12:58:43.849211', '2018-02-02 12:58:43.849712')
2018-02-02 12:58:43,876 INFO sqlalchemy.engine.base.Engine ROLLBACK
2018-02-02 12:58:43,917 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stock")
2018-02-02 12:58:43,920 INFO sqlalchemy.engine.base.Engine ()
2018-02-02 12:58:43,930 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-02-02 12:58:43,933 INFO sqlalchemy.engine.base.Engine INSERT INTO stock (code, name, category_id, activated, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?)
2018-02-

## 生SQLテスト 

In [7]:
rows = engine.execute('SELECT * FROM history')
# 表示
for row in rows:
    print(row)
rows = engine.execute('SELECT * FROM stock')
# 表示
for row in rows:
    print(row)

2018-02-02 12:58:44,098 INFO sqlalchemy.engine.base.Engine SELECT * FROM history
2018-02-02 12:58:44,100 INFO sqlalchemy.engine.base.Engine ()
(1301, '2000-01-04 00:00:00.000000', 1350.0, 1350.0, 1330.0, 1330.0, 163000)
(1301, '2000-01-05 00:00:00.000000', 1370.0, 1450.0, 1350.0, 1350.0, 218000)
(1301, '2000-01-06 00:00:00.000000', 1360.0, 1380.0, 1350.0, 1350.0, 194000)
(1301, '2000-01-07 00:00:00.000000', 1380.0, 1410.0, 1360.0, 1370.0, 211000)
(1301, '2000-01-11 00:00:00.000000', 1370.0, 1380.0, 1350.0, 1350.0, 236000)
(1301, '2000-01-12 00:00:00.000000', 1360.0, 1380.0, 1350.0, 1370.0, 251000)
(1301, '2000-01-13 00:00:00.000000', 1380.0, 1430.0, 1370.0, 1390.0, 183000)
(1301, '2000-01-14 00:00:00.000000', 1400.0, 1450.0, 1380.0, 1410.0, 493000)
(1301, '2000-01-17 00:00:00.000000', 1450.0, 1630.0, 1440.0, 1600.0, 391000)
(1301, '2000-01-18 00:00:00.000000', 1570.0, 1570.0, 1470.0, 1520.0, 195000)
(1301, '2000-01-19 00:00:00.000000', 1470.0, 1570.0, 1470.0, 1520.0, 173000)
(1301, '20

(1301, '2000-07-25 00:00:00.000000', 1450.0, 1510.0, 1450.0, 1510.0, 135000)
(1301, '2000-07-26 00:00:00.000000', 1500.0, 1500.0, 1460.0, 1460.0, 99000)
(1301, '2000-07-27 00:00:00.000000', 1480.0, 1500.0, 1450.0, 1480.0, 225000)
(1301, '2000-07-28 00:00:00.000000', 1480.0, 1490.0, 1450.0, 1460.0, 163000)
(1301, '2000-07-31 00:00:00.000000', 1450.0, 1460.0, 1420.0, 1420.0, 173000)
(1301, '2000-08-01 00:00:00.000000', 1410.0, 1430.0, 1400.0, 1420.0, 187000)
(1301, '2000-08-02 00:00:00.000000', 1430.0, 1450.0, 1430.0, 1450.0, 56000)
(1301, '2000-08-03 00:00:00.000000', 1420.0, 1450.0, 1420.0, 1420.0, 203000)
(1301, '2000-08-04 00:00:00.000000', 1430.0, 1440.0, 1410.0, 1440.0, 243000)
(1301, '2000-08-07 00:00:00.000000', 1420.0, 1450.0, 1420.0, 1440.0, 119000)
(1301, '2000-08-08 00:00:00.000000', 1430.0, 1440.0, 1410.0, 1420.0, 164000)
(1301, '2000-08-09 00:00:00.000000', 1430.0, 1450.0, 1410.0, 1450.0, 119000)
(1301, '2000-08-10 00:00:00.000000', 1430.0, 1460.0, 1430.0, 1460.0, 111000)
(

(1301, '2001-05-10 00:00:00.000000', 1550.0, 1580.0, 1550.0, 1550.0, 297000)
(1301, '2001-05-11 00:00:00.000000', 1570.0, 1580.0, 1560.0, 1570.0, 820000)
(1301, '2001-05-14 00:00:00.000000', 1580.0, 1580.0, 1540.0, 1540.0, 250000)
(1301, '2001-05-15 00:00:00.000000', 1540.0, 1570.0, 1520.0, 1560.0, 379000)
(1301, '2001-05-16 00:00:00.000000', 1560.0, 1560.0, 1520.0, 1520.0, 237000)
(1301, '2001-05-17 00:00:00.000000', 1540.0, 1540.0, 1500.0, 1510.0, 390000)
(1301, '2001-05-18 00:00:00.000000', 1530.0, 1550.0, 1510.0, 1520.0, 358000)
(1301, '2001-05-21 00:00:00.000000', 1520.0, 1540.0, 1520.0, 1540.0, 275000)
(1301, '2001-05-22 00:00:00.000000', 1530.0, 1540.0, 1510.0, 1510.0, 231000)
(1301, '2001-05-23 00:00:00.000000', 1530.0, 1540.0, 1510.0, 1520.0, 295000)
(1301, '2001-05-24 00:00:00.000000', 1520.0, 1540.0, 1510.0, 1530.0, 316000)
(1301, '2001-05-25 00:00:00.000000', 1520.0, 1540.0, 1510.0, 1530.0, 348000)
(1301, '2001-05-28 00:00:00.000000', 1520.0, 1530.0, 1500.0, 1500.0, 139000)

(1301, '2002-06-21 00:00:00.000000', 1310.0, 1320.0, 1300.0, 1300.0, 255000)
(1301, '2002-06-24 00:00:00.000000', 1290.0, 1300.0, 1280.0, 1280.0, 166000)
(1301, '2002-06-25 00:00:00.000000', 1280.0, 1320.0, 1280.0, 1320.0, 356000)
(1301, '2002-06-26 00:00:00.000000', 1300.0, 1310.0, 1280.0, 1280.0, 408000)
(1301, '2002-06-27 00:00:00.000000', 1300.0, 1310.0, 1280.0, 1310.0, 214000)
(1301, '2002-06-28 00:00:00.000000', 1320.0, 1330.0, 1300.0, 1330.0, 229000)
(1301, '2002-07-01 00:00:00.000000', 1310.0, 1330.0, 1300.0, 1300.0, 161000)
(1301, '2002-07-02 00:00:00.000000', 1300.0, 1320.0, 1300.0, 1320.0, 207000)
(1301, '2002-07-03 00:00:00.000000', 1320.0, 1360.0, 1310.0, 1360.0, 297000)
(1301, '2002-07-04 00:00:00.000000', 1340.0, 1360.0, 1340.0, 1340.0, 138000)
(1301, '2002-07-05 00:00:00.000000', 1350.0, 1390.0, 1350.0, 1370.0, 430000)
(1301, '2002-07-08 00:00:00.000000', 1370.0, 1380.0, 1330.0, 1330.0, 239000)
(1301, '2002-07-09 00:00:00.000000', 1350.0, 1380.0, 1340.0, 1360.0, 222000)

(1301, '2002-12-27 00:00:00.000000', 1150.0, 1170.0, 1140.0, 1150.0, 1027000)
(1301, '2002-12-30 00:00:00.000000', 1140.0, 1140.0, 1130.0, 1130.0, 271000)
(1301, '2003-01-06 00:00:00.000000', 1150.0, 1160.0, 1130.0, 1150.0, 361000)
(1301, '2003-01-07 00:00:00.000000', 1160.0, 1200.0, 1140.0, 1180.0, 1773000)
(1301, '2003-01-08 00:00:00.000000', 1190.0, 1230.0, 1180.0, 1220.0, 2096000)
(1301, '2003-01-09 00:00:00.000000', 1250.0, 1300.0, 1240.0, 1280.0, 4613000)
(1301, '2003-01-10 00:00:00.000000', 1280.0, 1300.0, 1250.0, 1270.0, 1766000)
(1301, '2003-01-14 00:00:00.000000', 1250.0, 1280.0, 1230.0, 1270.0, 910000)
(1301, '2003-01-15 00:00:00.000000', 1290.0, 1350.0, 1280.0, 1340.0, 3020000)
(1301, '2003-01-16 00:00:00.000000', 1320.0, 1360.0, 1310.0, 1350.0, 1769000)
(1301, '2003-01-17 00:00:00.000000', 1390.0, 1400.0, 1360.0, 1380.0, 2676000)
(1301, '2003-01-20 00:00:00.000000', 1380.0, 1390.0, 1340.0, 1350.0, 1182000)
(1301, '2003-01-21 00:00:00.000000', 1350.0, 1380.0, 1330.0, 1380.0

...

(1306, '2014-10-06 00:00:00.000000', 1327.0, 1333.0, 1321.0, 1324.0, 2569490)
(1306, '2014-10-07 00:00:00.000000', 1324.0, 1332.0, 1317.0, 1320.0, 2621030)
(1306, '2014-10-08 00:00:00.000000', 1299.0, 1307.0, 1296.0, 1305.0, 2262750)
(1306, '2014-10-09 00:00:00.000000', 1308.0, 1311.0, 1287.0, 1289.0, 2858940)
(1306, '2014-10-10 00:00:00.000000', 1265.0, 1273.0, 1263.0, 1270.0, 6167570)
(1306, '2014-10-14 00:00:00.000000', 1243.0, 1253.0, 1238.0, 1243.0, 5680320)
(1306, '2014-10-15 00:00:00.000000', 1247.0, 1253.0, 1238.0, 1252.0, 3258050)
(1306, '2014-10-16 00:00:00.000000', 1223.0, 1229.0, 1217.0, 1221.0, 14468620)
(1306, '2014-10-17 00:00:00.000000', 1225.0, 1227.0, 1202.0, 1204.0, 4947110)
(1306, '2014-10-20 00:00:00.000000', 1238.0, 1254.0, 1235.0, 1252.0, 6154960)
(1306, '2014-10-21 00:00:00.000000', 1255.0, 1255.0, 1227.0, 1233.0, 4163060)
(1306, '2014-10-22 00:00:00.000000', 1254.0, 1264.0, 1250.0, 1263.0, 2273640)
(1306, '2014-10-23 00:00:00.000000', 1258.0, 1265.0, 1250.0, 12

(1306, '2015-03-19 00:00:00.000000', 1621.0, 1625.0, 1601.0, 1613.0, 4527520)
(1306, '2015-03-20 00:00:00.000000', 1615.0, 1618.0, 1605.0, 1618.0, 2995200)
(1306, '2015-03-23 00:00:00.000000', 1622.0, 1632.0, 1620.0, 1630.0, 2865030)
(1306, '2015-03-24 00:00:00.000000', 1624.0, 1628.0, 1615.0, 1623.0, 3432290)
(1306, '2015-03-25 00:00:00.000000', 1626.0, 1630.0, 1612.0, 1629.0, 4308780)
(1306, '2015-03-26 00:00:00.000000', 1617.0, 1618.0, 1598.0, 1606.0, 4125030)
(1306, '2015-03-27 00:00:00.000000', 1602.0, 1623.0, 1583.0, 1595.0, 12531080)
(1306, '2015-03-30 00:00:00.000000', 1600.0, 1610.0, 1591.0, 1607.0, 5426240)
(1306, '2015-03-31 00:00:00.000000', 1626.0, 1628.0, 1590.0, 1590.0, 4219900)
(1306, '2015-04-01 00:00:00.000000', 1582.0, 1589.0, 1566.0, 1576.0, 6693490)
(1306, '2015-04-02 00:00:00.000000', 1584.0, 1614.0, 1583.0, 1601.0, 4634710)
(1306, '2015-04-03 00:00:00.000000', 1601.0, 1613.0, 1600.0, 1613.0, 2874400)
(1306, '2015-04-06 00:00:00.000000', 1601.0, 1611.0, 1596.0, 16

(1308, '2008-04-03 00:00:00.000000', 1313.0, 1325.0, 1307.0, 1325.0, 379000)
(1308, '2008-04-04 00:00:00.000000', 1315.0, 1320.0, 1311.0, 1315.0, 122000)
(1308, '2008-04-07 00:00:00.000000', 1317.0, 1335.0, 1309.0, 1330.0, 27000)
(1308, '2008-04-08 00:00:00.000000', 1325.0, 1325.0, 1306.0, 1308.0, 55000)
(1308, '2008-04-09 00:00:00.000000', 1311.0, 1313.0, 1280.0, 1288.0, 64000)
(1308, '2008-04-10 00:00:00.000000', 1272.0, 1283.0, 1267.0, 1274.0, 66000)
(1308, '2008-04-11 00:00:00.000000', 1281.0, 1303.0, 1281.0, 1303.0, 86000)
(1308, '2008-04-14 00:00:00.000000', 1263.0, 1273.0, 1263.0, 1272.0, 519000)
(1308, '2008-04-15 00:00:00.000000', 1272.0, 1284.0, 1266.0, 1281.0, 185000)
(1308, '2008-04-16 00:00:00.000000', 1296.0, 1303.0, 1294.0, 1298.0, 123000)
(1308, '2008-04-17 00:00:00.000000', 1323.0, 1332.0, 1317.0, 1317.0, 293000)
(1308, '2008-04-18 00:00:00.000000', 1322.0, 1332.0, 1313.0, 1332.0, 96000)
(1308, '2008-04-21 00:00:00.000000', 1359.0, 1360.0, 1350.0, 1358.0, 199000)
(1308

(1308, '2016-05-27 00:00:00.000000', 1383.0, 1389.0, 1379.0, 1385.0, 492200)
(1308, '2016-05-30 00:00:00.000000', 1396.0, 1402.0, 1389.0, 1400.0, 158800)
(1308, '2016-05-31 00:00:00.000000', 1395.0, 1416.0, 1395.0, 1413.0, 168600)
(1308, '2016-06-01 00:00:00.000000', 1403.0, 1410.0, 1394.0, 1398.0, 318200)
(1308, '2016-06-02 00:00:00.000000', 1388.0, 1388.0, 1364.0, 1365.0, 826900)
(1308, '2016-06-03 00:00:00.000000', 1371.0, 1375.0, 1365.0, 1373.0, 38800)
(1308, '2016-06-06 00:00:00.000000', 1350.0, 1367.0, 1345.0, 1366.0, 108400)
(1308, '2016-06-07 00:00:00.000000', 1372.0, 1377.0, 1365.0, 1375.0, 182300)
(1308, '2016-06-08 00:00:00.000000', 1379.0, 1386.0, 1368.0, 1385.0, 77700)
(1308, '2016-06-09 00:00:00.000000', 1377.0, 1382.0, 1367.0, 1371.0, 527700)
(1308, '2016-06-10 00:00:00.000000', 1366.0, 1369.0, 1357.0, 1365.0, 278100)
(1308, '2016-06-13 00:00:00.000000', 1339.0, 1340.0, 1318.0, 1319.0, 565000)
(1308, '2016-06-14 00:00:00.000000', 1316.0, 1321.0, 1297.0, 1305.0, 915700)
(

2018-02-02 12:58:47,132 INFO sqlalchemy.engine.base.Engine ()
(1301, '(株)極洋', '水産・農林業', 1, '2018-01-28 23:41:43.317769', '2018-01-28 23:41:43.318270')
(1305, 'ダイワ 上場投信-トピックス', 'ETF', 1, '2018-01-28 23:41:43.935182', '2018-01-28 23:41:43.935683')
(1306, 'TOPIX連動型上場投資信託', 'ETF', 1, '2018-01-28 23:41:44.539835', '2018-01-28 23:41:44.540336')
(1308, '上場インデックスファンドTOPIX', 'ETF', 1, '2018-01-28 23:41:45.004785', '2018-01-28 23:41:45.005286')


## 複雑なSQL

In [8]:
t = text("SELECT date FROM history")
rows = engine.execute(t)
for row in rows:
    print(row)

2018-02-02 12:58:47,148 INFO sqlalchemy.engine.base.Engine SELECT date FROM history
2018-02-02 12:58:47,149 INFO sqlalchemy.engine.base.Engine ()
('2000-01-04 00:00:00.000000',)
('2000-01-05 00:00:00.000000',)
('2000-01-06 00:00:00.000000',)
('2000-01-07 00:00:00.000000',)
('2000-01-11 00:00:00.000000',)
('2000-01-12 00:00:00.000000',)
('2000-01-13 00:00:00.000000',)
('2000-01-14 00:00:00.000000',)
('2000-01-17 00:00:00.000000',)
('2000-01-18 00:00:00.000000',)
('2000-01-19 00:00:00.000000',)
('2000-01-20 00:00:00.000000',)
('2000-01-21 00:00:00.000000',)
('2000-01-24 00:00:00.000000',)
('2000-01-25 00:00:00.000000',)
('2000-01-26 00:00:00.000000',)
('2000-01-27 00:00:00.000000',)
('2000-01-28 00:00:00.000000',)
('2000-01-31 00:00:00.000000',)
('2000-02-01 00:00:00.000000',)
('2000-02-02 00:00:00.000000',)
('2000-02-03 00:00:00.000000',)
('2000-02-04 00:00:00.000000',)
('2000-02-07 00:00:00.000000',)
('2000-02-08 00:00:00.000000',)
('2000-02-09 00:00:00.000000',)
('2000-02-10 00:00:00.

('2004-09-07 00:00:00.000000',)
('2004-09-08 00:00:00.000000',)
('2004-09-09 00:00:00.000000',)
('2004-09-10 00:00:00.000000',)
('2004-09-13 00:00:00.000000',)
('2004-09-14 00:00:00.000000',)
('2004-09-15 00:00:00.000000',)
('2004-09-16 00:00:00.000000',)
('2004-09-17 00:00:00.000000',)
('2004-09-21 00:00:00.000000',)
('2004-09-22 00:00:00.000000',)
('2004-09-24 00:00:00.000000',)
('2004-09-27 00:00:00.000000',)
('2004-09-28 00:00:00.000000',)
('2004-09-29 00:00:00.000000',)
('2004-09-30 00:00:00.000000',)
('2004-10-01 00:00:00.000000',)
('2004-10-04 00:00:00.000000',)
('2004-10-05 00:00:00.000000',)
('2004-10-06 00:00:00.000000',)
('2004-10-07 00:00:00.000000',)
('2004-10-08 00:00:00.000000',)
('2004-10-12 00:00:00.000000',)
('2004-10-13 00:00:00.000000',)
('2004-10-14 00:00:00.000000',)
('2004-10-15 00:00:00.000000',)
('2004-10-18 00:00:00.000000',)
('2004-10-19 00:00:00.000000',)
('2004-10-20 00:00:00.000000',)
('2004-10-21 00:00:00.000000',)
('2004-10-22 00:00:00.000000',)
('2004-1

('2012-08-24 00:00:00.000000',)
('2012-08-27 00:00:00.000000',)
('2012-08-28 00:00:00.000000',)
('2012-08-29 00:00:00.000000',)
('2012-08-30 00:00:00.000000',)
('2012-08-31 00:00:00.000000',)
('2012-09-03 00:00:00.000000',)
('2012-09-04 00:00:00.000000',)
('2012-09-05 00:00:00.000000',)
('2012-09-06 00:00:00.000000',)
('2012-09-07 00:00:00.000000',)
('2012-09-10 00:00:00.000000',)
('2012-09-11 00:00:00.000000',)
('2012-09-12 00:00:00.000000',)
('2012-09-13 00:00:00.000000',)
('2012-09-14 00:00:00.000000',)
('2012-09-18 00:00:00.000000',)
('2012-09-19 00:00:00.000000',)
('2012-09-20 00:00:00.000000',)
('2012-09-21 00:00:00.000000',)
('2012-09-24 00:00:00.000000',)
('2012-09-25 00:00:00.000000',)
('2012-09-26 00:00:00.000000',)
('2012-09-27 00:00:00.000000',)
('2012-09-28 00:00:00.000000',)
('2012-10-01 00:00:00.000000',)
('2012-10-02 00:00:00.000000',)
('2012-10-03 00:00:00.000000',)
('2012-10-04 00:00:00.000000',)
('2012-10-05 00:00:00.000000',)
('2012-10-09 00:00:00.000000',)
('2012-1

('2004-02-10 00:00:00.000000',)
('2004-02-12 00:00:00.000000',)
('2004-02-13 00:00:00.000000',)
('2004-02-16 00:00:00.000000',)
('2004-02-17 00:00:00.000000',)
('2004-02-18 00:00:00.000000',)
('2004-02-19 00:00:00.000000',)
('2004-02-20 00:00:00.000000',)
('2004-02-23 00:00:00.000000',)
('2004-02-24 00:00:00.000000',)
('2004-02-25 00:00:00.000000',)
('2004-02-26 00:00:00.000000',)
('2004-02-27 00:00:00.000000',)
('2004-03-01 00:00:00.000000',)
('2004-03-02 00:00:00.000000',)
('2004-03-03 00:00:00.000000',)
('2004-03-04 00:00:00.000000',)
('2004-03-05 00:00:00.000000',)
('2004-03-08 00:00:00.000000',)
('2004-03-09 00:00:00.000000',)
('2004-03-10 00:00:00.000000',)
('2004-03-11 00:00:00.000000',)
('2004-03-12 00:00:00.000000',)
('2004-03-15 00:00:00.000000',)
('2004-03-16 00:00:00.000000',)
('2004-03-17 00:00:00.000000',)
('2004-03-18 00:00:00.000000',)
('2004-03-19 00:00:00.000000',)
('2004-03-22 00:00:00.000000',)
('2004-03-23 00:00:00.000000',)
('2004-03-24 00:00:00.000000',)
('2004-0

('2012-08-01 00:00:00.000000',)
('2012-08-02 00:00:00.000000',)
('2012-08-03 00:00:00.000000',)
('2012-08-06 00:00:00.000000',)
('2012-08-07 00:00:00.000000',)
('2012-08-08 00:00:00.000000',)
('2012-08-09 00:00:00.000000',)
('2012-08-10 00:00:00.000000',)
('2012-08-13 00:00:00.000000',)
('2012-08-14 00:00:00.000000',)
('2012-08-15 00:00:00.000000',)
('2012-08-16 00:00:00.000000',)
('2012-08-17 00:00:00.000000',)
('2012-08-20 00:00:00.000000',)
('2012-08-21 00:00:00.000000',)
('2012-08-22 00:00:00.000000',)
('2012-08-23 00:00:00.000000',)
('2012-08-24 00:00:00.000000',)
('2012-08-27 00:00:00.000000',)
('2012-08-28 00:00:00.000000',)
('2012-08-29 00:00:00.000000',)
('2012-08-30 00:00:00.000000',)
('2012-08-31 00:00:00.000000',)
('2012-09-03 00:00:00.000000',)
('2012-09-04 00:00:00.000000',)
('2012-09-05 00:00:00.000000',)
('2012-09-06 00:00:00.000000',)
('2012-09-07 00:00:00.000000',)
('2012-09-10 00:00:00.000000',)
('2012-09-11 00:00:00.000000',)
('2012-09-12 00:00:00.000000',)
('2012-0

('2005-06-06 00:00:00.000000',)
('2005-06-07 00:00:00.000000',)
('2005-06-08 00:00:00.000000',)
('2005-06-09 00:00:00.000000',)
('2005-06-10 00:00:00.000000',)
('2005-06-13 00:00:00.000000',)
('2005-06-14 00:00:00.000000',)
('2005-06-15 00:00:00.000000',)
('2005-06-16 00:00:00.000000',)
('2005-06-17 00:00:00.000000',)
('2005-06-20 00:00:00.000000',)
('2005-06-21 00:00:00.000000',)
('2005-06-22 00:00:00.000000',)
('2005-06-23 00:00:00.000000',)
('2005-06-24 00:00:00.000000',)
('2005-06-27 00:00:00.000000',)
('2005-06-28 00:00:00.000000',)
('2005-06-29 00:00:00.000000',)
('2005-06-30 00:00:00.000000',)
('2005-07-01 00:00:00.000000',)
('2005-07-04 00:00:00.000000',)
('2005-07-05 00:00:00.000000',)
('2005-07-06 00:00:00.000000',)
('2005-07-07 00:00:00.000000',)
('2005-07-08 00:00:00.000000',)
('2005-07-11 00:00:00.000000',)
('2005-07-12 00:00:00.000000',)
('2005-07-13 00:00:00.000000',)
('2005-07-14 00:00:00.000000',)
('2005-07-15 00:00:00.000000',)
('2005-07-19 00:00:00.000000',)
('2005-0

('2013-03-18 00:00:00.000000',)
('2013-03-19 00:00:00.000000',)
('2013-03-21 00:00:00.000000',)
('2013-03-22 00:00:00.000000',)
('2013-03-25 00:00:00.000000',)
('2013-03-26 00:00:00.000000',)
('2013-03-27 00:00:00.000000',)
('2013-03-28 00:00:00.000000',)
('2013-03-29 00:00:00.000000',)
('2013-04-01 00:00:00.000000',)
('2013-04-02 00:00:00.000000',)
('2013-04-03 00:00:00.000000',)
('2013-04-04 00:00:00.000000',)
('2013-04-05 00:00:00.000000',)
('2013-04-08 00:00:00.000000',)
('2013-04-09 00:00:00.000000',)
('2013-04-10 00:00:00.000000',)
('2013-04-11 00:00:00.000000',)
('2013-04-12 00:00:00.000000',)
('2013-04-15 00:00:00.000000',)
('2013-04-16 00:00:00.000000',)
('2013-04-17 00:00:00.000000',)
('2013-04-18 00:00:00.000000',)
('2013-04-19 00:00:00.000000',)
('2013-04-22 00:00:00.000000',)
('2013-04-23 00:00:00.000000',)
('2013-04-24 00:00:00.000000',)
('2013-04-25 00:00:00.000000',)
('2013-04-26 00:00:00.000000',)
('2013-04-30 00:00:00.000000',)
('2013-05-01 00:00:00.000000',)
('2013-0

('2006-04-18 00:00:00.000000',)
('2006-04-19 00:00:00.000000',)
('2006-04-20 00:00:00.000000',)
('2006-04-21 00:00:00.000000',)
('2006-04-24 00:00:00.000000',)
('2006-04-25 00:00:00.000000',)
('2006-04-26 00:00:00.000000',)
('2006-04-27 00:00:00.000000',)
('2006-04-28 00:00:00.000000',)
('2006-05-01 00:00:00.000000',)
('2006-05-02 00:00:00.000000',)
('2006-05-08 00:00:00.000000',)
('2006-05-09 00:00:00.000000',)
('2006-05-10 00:00:00.000000',)
('2006-05-11 00:00:00.000000',)
('2006-05-12 00:00:00.000000',)
('2006-05-15 00:00:00.000000',)
('2006-05-16 00:00:00.000000',)
('2006-05-17 00:00:00.000000',)
('2006-05-18 00:00:00.000000',)
('2006-05-19 00:00:00.000000',)
('2006-05-22 00:00:00.000000',)
('2006-05-23 00:00:00.000000',)
('2006-05-24 00:00:00.000000',)
('2006-05-25 00:00:00.000000',)
('2006-05-26 00:00:00.000000',)
('2006-05-29 00:00:00.000000',)
('2006-05-30 00:00:00.000000',)
('2006-05-31 00:00:00.000000',)
('2006-06-01 00:00:00.000000',)
('2006-06-02 00:00:00.000000',)
('2006-0

('2015-03-16 00:00:00.000000',)
('2015-03-17 00:00:00.000000',)
('2015-03-18 00:00:00.000000',)
('2015-03-19 00:00:00.000000',)
('2015-03-20 00:00:00.000000',)
('2015-03-23 00:00:00.000000',)
('2015-03-24 00:00:00.000000',)
('2015-03-25 00:00:00.000000',)
('2015-03-26 00:00:00.000000',)
('2015-03-27 00:00:00.000000',)
('2015-03-30 00:00:00.000000',)
('2015-03-31 00:00:00.000000',)
('2015-04-01 00:00:00.000000',)
('2015-04-02 00:00:00.000000',)
('2015-04-03 00:00:00.000000',)
('2015-04-06 00:00:00.000000',)
('2015-04-07 00:00:00.000000',)
('2015-04-08 00:00:00.000000',)
('2015-04-09 00:00:00.000000',)
('2015-04-10 00:00:00.000000',)
('2015-04-13 00:00:00.000000',)
('2015-04-14 00:00:00.000000',)
('2015-04-15 00:00:00.000000',)
('2015-04-16 00:00:00.000000',)
('2015-04-17 00:00:00.000000',)
('2015-04-20 00:00:00.000000',)
('2015-04-21 00:00:00.000000',)
('2015-04-22 00:00:00.000000',)
('2015-04-23 00:00:00.000000',)
('2015-04-24 00:00:00.000000',)
('2015-04-27 00:00:00.000000',)
('2015-0

In [9]:
t = text("SELECT * FROM history WHERE date BETWEEN '2004/01/01' AND '2004/01/08'")
rows = engine.execute(t)
for row in rows:
    print(row)

2018-02-02 12:58:49,682 INFO sqlalchemy.engine.base.Engine SELECT * FROM history WHERE date BETWEEN '2004/01/01' AND '2004/01/08'
2018-02-02 12:58:49,684 INFO sqlalchemy.engine.base.Engine ()


## 動的SQL発行

In [10]:
start_date = "'2004/01/01'"
end_date = "'2004/01/08'"

上と同じSQLを発行するためには日付を二重に囲う必要がある。
また、 BETWWEN 日付 AND 日付　とするために空白も必要である。 

In [11]:
t = text("SELECT * FROM history WHERE date BETWEEN " + start_date + " AND " + end_date )
rows = engine.execute(t)
for row in rows:
    print(row)

2018-02-02 12:58:49,916 INFO sqlalchemy.engine.base.Engine SELECT * FROM history WHERE date BETWEEN '2004/01/01' AND '2004/01/08'
2018-02-02 12:58:49,922 INFO sqlalchemy.engine.base.Engine ()


## より簡潔な動的SQL(予定)

In [12]:
stocks = session.query(StockData)
for stock_res in stocks.all():
    print(stock_res.code)

2018-02-02 12:58:50,019 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-02-02 12:58:50,024 INFO sqlalchemy.engine.base.Engine SELECT stock.code AS stock_code, stock.name AS stock_name, stock.category_id AS stock_category_id, stock.activated AS stock_activated, stock.created_at AS stock_created_at, stock.updated_at AS stock_updated_at 
FROM stock
2018-02-02 12:58:50,026 INFO sqlalchemy.engine.base.Engine ()
1301
1305
1306
1308


In [13]:
histories = session.query(History)
for his in histories.all():
    print(his.date)

2018-02-02 12:58:50,115 INFO sqlalchemy.engine.base.Engine SELECT history.code AS history_code, history.date AS history_date, history."Open" AS "history_Open", history."High" AS "history_High", history."Low" AS "history_Low", history."Close" AS "history_Close", history."Volume" AS "history_Volume" 
FROM history
2018-02-02 12:58:50,116 INFO sqlalchemy.engine.base.Engine ()
2000-01-04
2000-01-05
2000-01-06
2000-01-07
2000-01-11
2000-01-12
2000-01-13
2000-01-14
2000-01-17
2000-01-18
2000-01-19
2000-01-20
2000-01-21
2000-01-24
2000-01-25
2000-01-26
2000-01-27
2000-01-28
2000-01-31
2000-02-01
2000-02-02
2000-02-03
2000-02-04
2000-02-07
2000-02-08
2000-02-09
2000-02-10
2000-02-14
2000-02-15
2000-02-16
2000-02-17
2000-02-18
2000-02-21
2000-02-22
2000-02-23
2000-02-24
2000-02-25
2000-02-28
2000-02-29
2000-03-01
2000-03-02
2000-03-03
2000-03-06
2000-03-07
2000-03-08
2000-03-09
2000-03-10
2000-03-13
2000-03-14
2000-03-15
2000-03-16
2000-03-17
2000-03-21
2000-03-22
2000-03-23
2000-03-24
2000-03-2

2006-10-10
2006-10-11
2006-10-12
2006-10-13
2006-10-16
2006-10-17
2006-10-18
2006-10-19
2006-10-20
2006-10-23
2006-10-24
2006-10-25
2006-10-26
2006-10-27
2006-10-30
2006-10-31
2006-11-01
2006-11-02
2006-11-06
2006-11-07
2006-11-08
2006-11-09
2006-11-10
2006-11-13
2006-11-14
2006-11-15
2006-11-16
2006-11-17
2006-11-20
2006-11-21
2006-11-22
2006-11-24
2006-11-27
2006-11-28
2006-11-29
2006-11-30
2006-12-01
2006-12-04
2006-12-05
2006-12-06
2006-12-07
2006-12-08
2006-12-11
2006-12-12
2006-12-13
2006-12-14
2006-12-15
2006-12-18
2006-12-19
2006-12-20
2006-12-21
2006-12-22
2006-12-25
2006-12-26
2006-12-27
2006-12-28
2006-12-29
2007-01-04
2007-01-05
2007-01-09
2007-01-10
2007-01-11
2007-01-12
2007-01-15
2007-01-16
2007-01-17
2007-01-18
2007-01-19
2007-01-22
2007-01-23
2007-01-24
2007-01-25
2007-01-26
2007-01-29
2007-01-30
2007-01-31
2007-02-01
2007-02-02
2007-02-05
2007-02-06
2007-02-07
2007-02-08
2007-02-09
2007-02-13
2007-02-14
2007-02-15
2007-02-16
2007-02-19
2007-02-20
2007-02-21
2007-02-22

2016-05-18
2016-05-19
2016-05-20
2016-05-23
2016-05-24
2016-05-25
2016-05-26
2016-05-27
2016-05-30
2016-05-31
2016-06-01
2016-06-02
2016-06-03
2016-06-06
2016-06-07
2016-06-08
2016-06-09
2016-06-10
2016-06-13
2016-06-14
2016-06-15
2016-06-16
2016-06-17
2016-06-20
2016-06-21
2016-06-22
2016-06-23
2016-06-24
2016-06-27
2016-06-28
2016-06-29
2016-06-30
2016-07-01
2016-07-04
2016-07-05
2016-07-06
2016-07-07
2016-07-08
2016-07-11
2016-07-12
2016-07-13
2016-07-14
2016-07-15
2016-07-19
2016-07-20
2016-07-21
2016-07-22
2016-07-25
2016-07-26
2016-07-27
2016-07-28
2016-07-29
2016-08-01
2016-08-02
2016-08-03
2016-08-04
2016-08-05
2016-08-08
2016-08-09
2016-08-10
2016-08-12
2016-08-15
2016-08-16
2016-08-17
2016-08-18
2016-08-19
2016-08-22
2016-08-23
2016-08-24
2016-08-25
2016-08-26
2016-08-29
2016-08-30
2016-08-31
2016-09-01
2016-09-02
2016-09-05
2016-09-06
2016-09-07
2016-09-08
2016-09-09
2016-09-12
2016-09-13
2016-09-14
2016-09-15
2016-09-16
2016-09-20
2016-09-21
2016-09-23
2016-09-26
2016-09-27

2010-01-04
2010-01-05
2010-01-06
2010-01-07
2010-01-08
2010-01-12
2010-01-13
2010-01-14
2010-01-15
2010-01-18
2010-01-19
2010-01-20
2010-01-21
2010-01-22
2010-01-25
2010-01-26
2010-01-27
2010-01-28
2010-01-29
2010-02-01
2010-02-02
2010-02-03
2010-02-04
2010-02-05
2010-02-08
2010-02-09
2010-02-10
2010-02-12
2010-02-15
2010-02-16
2010-02-17
2010-02-18
2010-02-19
2010-02-22
2010-02-23
2010-02-24
2010-02-25
2010-02-26
2010-03-01
2010-03-02
2010-03-03
2010-03-04
2010-03-05
2010-03-08
2010-03-09
2010-03-10
2010-03-11
2010-03-12
2010-03-15
2010-03-16
2010-03-17
2010-03-18
2010-03-19
2010-03-23
2010-03-24
2010-03-25
2010-03-26
2010-03-29
2010-03-30
2010-03-31
2010-04-01
2010-04-02
2010-04-05
2010-04-06
2010-04-07
2010-04-08
2010-04-09
2010-04-12
2010-04-13
2010-04-14
2010-04-15
2010-04-16
2010-04-19
2010-04-20
2010-04-21
2010-04-22
2010-04-23
2010-04-26
2010-04-27
2010-04-28
2010-04-30
2010-05-06
2010-05-07
2010-05-10
2010-05-11
2010-05-12
2010-05-13
2010-05-14
2010-05-17
2010-05-18
2010-05-19

2002-03-12
2002-03-13
2002-03-14
2002-03-15
2002-03-18
2002-03-19
2002-03-20
2002-03-22
2002-03-25
2002-03-26
2002-03-27
2002-03-28
2002-03-29
2002-04-01
2002-04-02
2002-04-03
2002-04-04
2002-04-05
2002-04-08
2002-04-09
2002-04-10
2002-04-11
2002-04-12
2002-04-15
2002-04-16
2002-04-17
2002-04-18
2002-04-19
2002-04-22
2002-04-23
2002-04-24
2002-04-25
2002-04-26
2002-04-30
2002-05-01
2002-05-02
2002-05-07
2002-05-08
2002-05-09
2002-05-10
2002-05-13
2002-05-14
2002-05-15
2002-05-16
2002-05-17
2002-05-20
2002-05-21
2002-05-22
2002-05-23
2002-05-24
2002-05-27
2002-05-28
2002-05-29
2002-05-30
2002-05-31
2002-06-03
2002-06-04
2002-06-05
2002-06-06
2002-06-07
2002-06-10
2002-06-11
2002-06-12
2002-06-13
2002-06-14
2002-06-17
2002-06-18
2002-06-19
2002-06-20
2002-06-21
2002-06-24
2002-06-25
2002-06-26
2002-06-27
2002-06-28
2002-07-01
2002-07-02
2002-07-03
2002-07-04
2002-07-05
2002-07-08
2002-07-09
2002-07-10
2002-07-11
2002-07-12
2002-07-15
2002-07-16
2002-07-17
2002-07-18
2002-07-19
2002-07-22

2011-07-11
2011-07-12
2011-07-13
2011-07-14
2011-07-15
2011-07-19
2011-07-20
2011-07-21
2011-07-22
2011-07-25
2011-07-26
2011-07-27
2011-07-28
2011-07-29
2011-08-01
2011-08-02
2011-08-03
2011-08-04
2011-08-05
2011-08-08
2011-08-09
2011-08-10
2011-08-11
2011-08-12
2011-08-15
2011-08-16
2011-08-17
2011-08-18
2011-08-19
2011-08-22
2011-08-23
2011-08-24
2011-08-25
2011-08-26
2011-08-29
2011-08-30
2011-08-31
2011-09-01
2011-09-02
2011-09-05
2011-09-06
2011-09-07
2011-09-08
2011-09-09
2011-09-12
2011-09-13
2011-09-14
2011-09-15
2011-09-16
2011-09-20
2011-09-21
2011-09-22
2011-09-26
2011-09-27
2011-09-28
2011-09-29
2011-09-30
2011-10-03
2011-10-04
2011-10-05
2011-10-06
2011-10-07
2011-10-11
2011-10-12
2011-10-13
2011-10-14
2011-10-17
2011-10-18
2011-10-19
2011-10-20
2011-10-21
2011-10-24
2011-10-25
2011-10-26
2011-10-27
2011-10-28
2011-10-31
2011-11-01
2011-11-02
2011-11-04
2011-11-07
2011-11-08
2011-11-09
2011-11-10
2011-11-11
2011-11-14
2011-11-15
2011-11-16
2011-11-17
2011-11-18
2011-11-21

2005-04-21
2005-04-22
2005-04-25
2005-04-26
2005-04-27
2005-04-28
2005-05-02
2005-05-06
2005-05-09
2005-05-10
2005-05-11
2005-05-12
2005-05-13
2005-05-16
2005-05-17
2005-05-18
2005-05-19
2005-05-20
2005-05-23
2005-05-24
2005-05-25
2005-05-26
2005-05-27
2005-05-30
2005-05-31
2005-06-01
2005-06-02
2005-06-03
2005-06-06
2005-06-07
2005-06-08
2005-06-09
2005-06-10
2005-06-13
2005-06-14
2005-06-15
2005-06-16
2005-06-17
2005-06-20
2005-06-21
2005-06-22
2005-06-23
2005-06-24
2005-06-27
2005-06-28
2005-06-29
2005-06-30
2005-07-01
2005-07-04
2005-07-05
2005-07-06
2005-07-07
2005-07-08
2005-07-11
2005-07-12
2005-07-13
2005-07-14
2005-07-15
2005-07-19
2005-07-20
2005-07-21
2005-07-22
2005-07-25
2005-07-26
2005-07-27
2005-07-28
2005-07-29
2005-08-01
2005-08-02
2005-08-03
2005-08-04
2005-08-05
2005-08-08
2005-08-09
2005-08-10
2005-08-11
2005-08-12
2005-08-15
2005-08-16
2005-08-17
2005-08-18
2005-08-19
2005-08-22
2005-08-23
2005-08-24
2005-08-25
2005-08-26
2005-08-29
2005-08-30
2005-08-31
2005-09-01

2015-03-23
2015-03-24
2015-03-25
2015-03-26
2015-03-27
2015-03-30
2015-03-31
2015-04-01
2015-04-02
2015-04-03
2015-04-06
2015-04-07
2015-04-08
2015-04-09
2015-04-10
2015-04-13
2015-04-14
2015-04-15
2015-04-16
2015-04-17
2015-04-20
2015-04-21
2015-04-22
2015-04-23
2015-04-24
2015-04-27
2015-04-28
2015-04-30
2015-05-01
2015-05-07
2015-05-08
2015-05-11
2015-05-12
2015-05-13
2015-05-14
2015-05-15
2015-05-18
2015-05-19
2015-05-20
2015-05-21
2015-05-22
2015-05-25
2015-05-26
2015-05-27
2015-05-28
2015-05-29
2015-06-01
2015-06-02
2015-06-03
2015-06-04
2015-06-05
2015-06-08
2015-06-09
2015-06-10
2015-06-11
2015-06-12
2015-06-15
2015-06-16
2015-06-17
2015-06-18
2015-06-19
2015-06-22
2015-06-23
2015-06-24
2015-06-25
2015-06-26
2015-06-29
2015-06-30
2015-07-01
2015-07-02
2015-07-03
2015-07-06
2015-07-07
2015-07-08
2015-07-09
2015-07-10
2015-07-13
2015-07-14
2015-07-15
2015-07-16
2015-07-17
2015-07-21
2015-07-22
2015-07-23
2015-07-24
2015-07-27
2015-07-28
2015-07-29
2015-07-30
2015-07-31
2015-08-03

In [22]:
start_date = datetime(2003,12,30)   #"'2004/01/01'"
end_date = datetime(2004,1,8)     #"'2004/01/08'"
histories = session.query(History)
for his in histories.filter(History.date >= start_date, History.date <= end_date).all():
    print(his.date)

2018-02-02 13:12:09,341 INFO sqlalchemy.engine.base.Engine SELECT history.code AS history_code, history.date AS history_date, history."Open" AS "history_Open", history."High" AS "history_High", history."Low" AS "history_Low", history."Close" AS "history_Close", history."Volume" AS "history_Volume" 
FROM history 
WHERE history.date >= ? AND history.date <= ?
2018-02-02 13:12:09,343 INFO sqlalchemy.engine.base.Engine ('2003-12-30 00:00:00.000000', '2004-01-08 00:00:00.000000')
2003-12-30
2004-01-05
2004-01-06
2004-01-07
2004-01-08
2003-12-30
2004-01-05
2004-01-06
2004-01-07
2004-01-08
2003-12-30
2004-01-05
2004-01-06
2004-01-07
2004-01-08
2003-12-30
2004-01-05
2004-01-06
2004-01-07
2004-01-08
